In [1]:
import torch
from simulator import blogm, bSqc, Neg, Sa
%run 'utils.py'

## load test result

In [15]:
# MPO no ancilla

seed = 0

mps_out = []
for N in range(4,36,2):
    l = torch.load(f'MPO/seed{seed}/record/mpo_N={N}_na.pt')
    mps_out.append([N, torch.tensor(l['test Sqc']).view(-1,200).mean(-1)[-1].item(), torch.tensor(l['test Neg']).view(-1,200).mean(-1)[-1].item(), torch.tensor(l['test Sa']).view(-1,200).mean(-1)[-1].item()])
mps_out

[[4, 0.364736407995224, 0.4148503541946411, 0.6909874081611633],
 [6, 0.40908727049827576, 0.4030132293701172, 0.6922643184661865],
 [8, 0.5202898979187012, 0.3697245419025421, 0.6928110718727112],
 [10, 0.6350785493850708, 0.32931357622146606, 0.6939518451690674],
 [12, 0.7240471839904785, 0.2949746251106262, 0.6931992173194885],
 [14, 0.7473224401473999, 0.28529077768325806, 0.6953920125961304],
 [16, 0.8022655248641968, 0.262577623128891, 0.6955318450927734],
 [18, 0.8470113277435303, 0.24008946120738983, 0.6964192390441895],
 [20, 0.9177682995796204, 0.20551328361034393, 0.695112943649292],
 [22, 0.9770315289497375, 0.1711874008178711, 0.6958082318305969],
 [24, 1.0242738723754883, 0.14908725023269653, 0.6973040103912354],
 [26, 1.053487777709961, 0.13219453394412994, 0.6959724426269531],
 [28, 1.0787769556045532, 0.11860309541225433, 0.6969874501228333],
 [30, 1.09284245967865, 0.106590636074543, 0.6972158551216125],
 [32, 1.1137124300003052, 0.0917612835764885, 0.6957629919052124

In [16]:
torch.save(mps_out, 'mps_out.pt')

In [17]:
# GPT no ancilla

seed = 1

gpt_out = []
for N in range(4,36,2):
    l = torch.load(f'GPT_together/seed{seed}/record/gpt_N={N}_na.pt')
    gpt_out.append([N, torch.tensor(l['test Sqc']).view(-1,200).mean(-1)[-1].item(), torch.tensor(l['test Neg']).view(-1,200).mean(-1)[-1].item(), torch.tensor(l['test Sa']).view(-1,200).mean(-1)[-1].item()])
gpt_out

[[4, 0.37280037999153137, 0.4131636321544647, 0.6908037066459656],
 [6, 0.4102906882762909, 0.4040994644165039, 0.6921830773353577],
 [8, 0.5034903883934021, 0.37162959575653076, 0.6910855770111084],
 [10, 0.6337853670120239, 0.3275477886199951, 0.6922305822372437],
 [12, 0.7163309454917908, 0.29513853788375854, 0.6901062726974487],
 [14, 0.7441816926002502, 0.28241536021232605, 0.6907975077629089],
 [16, 0.8188619017601013, 0.25158053636550903, 0.6924182772636414],
 [18, 0.8491512537002563, 0.2365495264530182, 0.6923715472221375],
 [20, 0.9247498512268066, 0.19789540767669678, 0.6917109489440918],
 [22, 0.9647826552391052, 0.17572282254695892, 0.6921895742416382],
 [24, 1.0269538164138794, 0.14290118217468262, 0.6924147009849548],
 [26, 1.0608333349227905, 0.12394537031650543, 0.69263756275177],
 [28, 1.0915954113006592, 0.10315950214862823, 0.6919042468070984],
 [30, 1.110666275024414, 0.08745766431093216, 0.6927810907363892],
 [32, 1.1533000469207764, 0.06018567085266113, 0.69180572

In [18]:
torch.save(gpt_out, 'gpt_out.pt')

In [19]:
# post select

seed = 0
train_size = 800000

post_out = []
for N in range(4,36,2):
    torch.manual_seed(seed)
    prepseq, shadow_state, rhoS = torch_data(f'data/data_{N}na.pickle', shuffle=True)
    prepseq, shadow_state, rhoS = prepseq[train_size:], shadow_state[train_size:], rhoS[train_size:]
    for i in range(2):
        for j in range(2):
            idx = (prepseq[:,::2].sum(-1)%2 == i)*(prepseq[:,1::2].sum(-1)%2 == j)
            rhoC = rhoS[idx].mean(0, keepdim=True)
            post_out.append([N, bSqc(rhoC,rhoC).item(), Neg(rhoC, rhoC).item(), Sa(rhoC, rhoC).item()])
post_out = torch.tensor(post_out).view(-1, 4, 4).mean(1).tolist()
post_out

NameError: name 'torch_data' is not defined

In [17]:
torch.save(post_out, 'post_out.pt')

## prepare N-mixed data

In [3]:
seed = 0
prepseq_all, shadow_state_all, rhoS_all = [], [], []
for N in range(4,36,2):
    torch.manual_seed(seed)
    prepseq, shadow_state, rhoS = torch_data(f'data/data_{N}na.pickle', shuffle=True)
    prepseq = torch.cat([prepseq+2, torch.zeros(prepseq.shape[0], 32-prepseq.shape[1], dtype=prepseq.dtype), torch.ones(prepseq.shape[0], 1, dtype=prepseq.dtype)], 1)
    prepseq_all.append(prepseq[None,...])
    shadow_state_all.append(shadow_state[None,...])
    rhoS_all.append(rhoS[None,...])
prepseq_all = torch.cat(prepseq_all)
shadow_state_all = torch.cat(shadow_state_all)
rhoS_all = torch.cat(rhoS_all)

In [4]:
prepseq_all.shape

torch.Size([16, 900000, 33])

In [5]:
torch.save(prepseq_all, f'data/prepseq_all.pt')
torch.save(shadow_state_all, f'data/shadow_state_all.pt')
torch.save(rhoS_all, f'data/rhoS_all.pt')

In [2]:
prepseq_all = torch.load(f'data/prepseq_all.pt')
shadow_state_all = torch.load(f'data/shadow_state_all.pt')
rhoS_all = torch.load(f'data/rhoS_all.pt')

In [3]:
train_ratio = 8/9

train_size = int(prepseq_all.shape[1]*train_ratio)
test_size = prepseq_all.shape[1]-train_size

prepseq_train, prepseq_test = prepseq_all[:,:train_size], prepseq_all[:,train_size:]
shadow_state_train, shadow_state_test = shadow_state_all[:,:train_size], shadow_state_all[:,train_size:]
rhoS_train, rhoS_test = rhoS_all[:,:train_size], rhoS_all[:,train_size:]

In [5]:
seed = 0
torch.manual_seed(seed)
prepseq_train, shadow_state_train, rhoS_train = shuffle(prepseq_train.flatten(0,1), shadow_state_train.flatten(0,1), rhoS_train.flatten(0,1))

In [6]:
torch.save(prepseq_train, f'data/prepseq_train.pt')
torch.save(shadow_state_train, f'data/shadow_state_train.pt')
torch.save(rhoS_train, f'data/rhoS_train.pt')

In [7]:
torch.save(prepseq_test, f'data/prepseq_test.pt')
torch.save(shadow_state_test, f'data/shadow_state_test.pt')
torch.save(rhoS_test, f'data/rhoS_test.pt')

## mean and std

In [ ]:
# MPO no ancilla

seed = 0

mps_out = []
for N in range(4,36,2):
    l = torch.load(f'MPO/seed{seed}/record/mpo_N={N}_na.pt')
    mps_out.append([N, torch.tensor(l['test Sqc']).view(-1,200).mean(-1)[-1].item(), torch.tensor(l['test Neg']).view(-1,200).mean(-1)[-1].item(), torch.tensor(l['test Sa']).view(-1,200).mean(-1)[-1].item()])
mps_out

In [1]:
import torch

# Initialize lists to store all values for each N
mps_N_values = []  # For Neg values
mps_S_values = []  # For Sqc values
mps_I_values = []  # For Sa values

# Collect all values for each N
for N in range(4,36,2):
    l = torch.load(f'MPO/seed0/record/mpo_N={N}_na.pt')
    # Get the last 200 values for each metric
    neg_values = torch.tensor(l['test Neg']).view(-1,200)[-1]
    sqc_values = torch.tensor(l['test Sqc']).view(-1,200)[-1]
    sa_values = torch.tensor(l['test Sa']).view(-1,200)[-1]
    
    mps_N_values.append(neg_values)
    mps_S_values.append(sqc_values)
    mps_I_values.append(sa_values)

# Convert to tensors
mps_N_values = torch.stack(mps_N_values)
mps_S_values = torch.stack(mps_S_values)
mps_I_values = torch.stack(mps_I_values)

# Calculate means
mps_N_mean = mps_N_values.mean(dim=1)
mps_S_mean = mps_S_values.mean(dim=1)
mps_I_mean = mps_I_values.mean(dim=1)

# Calculate standard deviations
# mps_N_std = torch.sqrt(mps_N_values.var(dim=1))
# mps_S_std = torch.sqrt(mps_S_values.var(dim=1))
# mps_I_std = torch.sqrt(mps_I_values.var(dim=1))

# Save the results
# torch.save(mps_N_mean, 'mps_N_mean.pt')
# torch.save(mps_N_std, 'mps_N_std.pt')
# torch.save(mps_S_mean, 'mps_S_mean.pt')
# torch.save(mps_S_std, 'mps_S_std.pt')
# torch.save(mps_I_mean, 'mps_I_mean.pt')
# torch.save(mps_I_std, 'mps_I_std.pt')

# Print the results
print('Means:')
print('N (Neg) mean:', mps_N_mean.tolist())
print('S (Sqc) mean:', mps_S_mean.tolist())
print('I (Sa) mean:', mps_I_mean.tolist())
print('\nStandard Deviations:')
print('N (Neg) std:', mps_N_std.tolist())
print('S (Sqc) std:', mps_S_std.tolist())
print('I (Sa) std:', mps_I_std.tolist())

Means:
N (Neg) mean: [0.4148503541946411, 0.4030132293701172, 0.3697245419025421, 0.32931357622146606, 0.2949746251106262, 0.28529077768325806, 0.262577623128891, 0.24008946120738983, 0.20551328361034393, 0.1711874008178711, 0.14908725023269653, 0.13219453394412994, 0.11860309541225433, 0.106590636074543, 0.0917612835764885, 0.06791458278894424]
S (Sqc) mean: [0.364736407995224, 0.40908727049827576, 0.5202898979187012, 0.6350785493850708, 0.7240471839904785, 0.7473224401473999, 0.8022655248641968, 0.8470113277435303, 0.9177682995796204, 0.9770315289497375, 1.0242738723754883, 1.053487777709961, 1.0787769556045532, 1.09284245967865, 1.1137124300003052, 1.1425203084945679]
I (Sa) mean: [0.6909874081611633, 0.6922643184661865, 0.6928110718727112, 0.6939518451690674, 0.6931992173194885, 0.6953920125961304, 0.6955318450927734, 0.6964192390441895, 0.695112943649292, 0.6958082318305969, 0.6973040103912354, 0.6959724426269531, 0.6969874501228333, 0.6972158551216125, 0.6957629919052124, 0.69590

In [4]:
mps_N_values.shape

torch.Size([16, 200])

In [23]:
len(torch.load(f'GPT_together/seed{0}/record/gpt_N={34}_na.pt')['test Neg'])

600